## **Cement Strength Prediction Model**
### **Author : Sujal Karbhari**

In [1]:
# Import Data Manipulation Library
import pandas as pd
import numpy as np

# Import Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Import Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,RobustScaler,LabelEncoder,OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,BaggingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [11]:
# Data Ingestion

def data_ingestion():
    try:
        df = pd.read_csv(r'C:\CementStrength_PredictionModel\data\raw\Concrete_Compressive_Strength.csv')
        print("Data Ingestion Successful")
    except:
        print("Data Ingestion Unsuccessful")
    return df

In [13]:
# data exploration
from collections import OrderedDict
numerical_stats = []

Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
LW = Q1 - 1.5 * IQR
UW = Q3 + 1.5 * IQR
Outlier_Count = ((df < LW) | (df > UW)).sum()
Outlier_Percentage = (Outlier_Count / df.shape[0]) * 100

def data_exploration(df):
    for i in df:
        
        num_stats = OrderedDict({
            "features": i,
            "Maximum": df[i].max(),
            "Minimum": df[i].min(),
            "Mean": df[i].mean(),
            "Median": df[i].median(),
            "Q1": Q1[i],
            "Q3": Q3[i],
            "IQR": IQR[i],
            "Lower Whisker": LW[i],
            "Upper Whisker": UW[i],
            "Outlier Count": Outlier_Count[i],
            "Outlier Percentage": Outlier_Percentage[i],
            "Standard Deviation": df[i].std(),
            "Skewness": df[i].skew(),
            "Kurtosis": df[i].kurtosis()
            })
        numerical_stats.append(num_stats)
        numerical_stats_report = pd.DataFrame(numerical_stats)
    return numerical_stats_report

numerical_stats_report = data_exploration(df)
numerical_stats_report

,features,Maximum,Minimum,Mean,Median,Q1,Q3,IQR,Lower Whisker,Upper Whisker,Outlier Count,Outlier Percentage,Standard Deviation,Skewness,Kurtosis
0,cement,540.000000,102.000000,281.167864,272.900000,192.375000,350.000000,157.625000,-44.062500,586.437500,0,0.000000,104.506364,0.509481,-0.520652
1,slag,359.400000,0.000000,73.895825,22.000000,0.000000,142.950000,142.950000,-214.425000,357.375000,2,0.194175,86.279342,0.800717,-0.508175
2,ash,200.100000,0.000000,54.188350,0.000000,0.000000,118.300000,118.300000,-177.450000,295.750000,0,0.000000,63.997004,0.537354,-1.328746
3,water,247.000000,121.800000,181.567282,185.000000,164.900000,192.000000,27.100000,124.250000,232.650000,9,0.873786,21.354219,0.074628,0.122082
4,superplastic,32.200000,0.000000,6.204660,6.400000,0.000000,10.200000,10.200000,-15.300000,25.500000,10,0.970874,5.973841,0.907203,1.411269
5,coarseagg,1145.000000,801.000000,972.918932,968.000000,932.000000,1029.400000,97.400000,785.900000,1175.500000,0,0.000000,77.753954,-0.040220,-0.599016
6,fineagg,992.600000,594.000000,773.580485,779.500000,730.950000,824.000000,93.050000,591.375000,963.575000,5,0.485437,80.175980,-0.253010,-0.102177
7,age,365.000000,1.000000,45.662136,28.000000,7.000000,56.000000,49.000000,-66.500000,129.500000,59,5.728155,63.169912,3.269177,12.168989
8,strength,82.600000,2.330000,35.817961,34.445000,23.710000,46.135000,22.425000,-9.927500,79.772500,4,0.388350,16.705742,0.416977,-0.313725
9,water_cement_ratio,1.882334,0.266892,0.748263,0.675346,0.533332,0.935160,0.401828,-0.069409,1.537901,18,1.747573,0.314003,0.958065,0.734109


In [14]:
# Dataset Information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   cement                    1030 non-null   float64
 1   slag                      1030 non-null   float64
 2   ash                       1030 non-null   float64
 3   water                     1030 non-null   float64
 4   superplastic              1030 non-null   float64
 5   coarseagg                 1030 non-null   float64
 6   fineagg                   1030 non-null   float64
 7   age                       1030 non-null   int64  
 8   strength                  1030 non-null   float64
 9   water_cement_ratio        1030 non-null   float64
 10  total_binder              1030 non-null   float64
 11  aggregate_to_cement       1030 non-null   float64
 12  cement_water_interaction  1030 non-null   float64
 13  age_strength_proxy        1030 non-null   float64
dtypes: float

In [16]:
# Data Preprocessing

def data_preprocessing(df):
        
    X = df.drop(columns = 'strength',)
    y = df['strength']

    # Use Train Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                            test_size=0.3,
                                                            random_state=1)
        
    # Use Scaling Techniques
    rs = RobustScaler().fit(X_train,y_train)
    X_train= rs.fit_transform(X_train)
    X_test= rs.transform(X_test)
        
    return X_train, X_test, y_train, y_test

In [18]:
# Model Building
def model_building(Xdf):
    
    models = {
        "Linear Regression": LinearRegression(),
        "Decision Tree": DecisionTreeRegressor(),
        "Random Forest": RandomForestRegressor(),
        "Gradient Boosting": GradientBoostingRegressor(),
        "Bagging Regressor": BaggingRegressor(),
        "AdaBoost Regressor": AdaBoostRegressor(),
        "Support Vector Regressor": SVR(),
        "K-Neighbors Regressor": KNeighborsRegressor()
        }
    
    return models

In [22]:
# Model Evaluation
def model_evaluation(models, X_train, X_test, y_train, y_test):
    r2scores = {}

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2scores[model_name] = r2_score(y_test, y_pred)
    return r2scores

In [23]:
r2scores = model_evaluation(models, X_train, X_test, y_train, y_test)
r2scores

{'Linear Regression': 0.8173690536956675,
 'Decision Tree': 0.8578198765881846,
 'Random Forest': 0.9096365295022435,
 'Gradient Boosting': 0.9084657510517422,
 'Bagging Regressor': 0.8987873538371791,
 'AdaBoost Regressor': 0.7973356980827431,
 'Support Vector Regressor': 0.7671012945149633,
 'K-Neighbors Regressor': 0.8077377472879232}